In [1]:
import sparkbeyond._api2.classes as sb
from sparkbeyond.predictionserver.api import PredictionServerClient
import pickle
import argparse
import urllib3
import json
import logging
import tqdm
import os
import pandas as pd
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
with open('SB_Credential.json', 'rb') as b:
    sb_config = json.load(b)

dp_credential = sb_config['DP']
client = sb.SparkBeyondClient(base_url = dp_credential['base_url'], api_key= dp_credential['api_key'],
                              verify_ssl_certificate=False)

eps_credential = sb_config['EPS']
eps_client = PredictionServerClient(url=eps_credential['base_url'], refresh_token=eps_credential['api_key'], validate_ssl_certificate=False)

INFO:sparkbeyond.predictionserver.api.client:Connection with https://192.168.106.199/ - Established


In [3]:
predict_df = pd.read_csv('preprocessed_data/nphr_03272023/train_unit1.csv')
context_df = pd.read_csv('preprocessed_data/nphr_03272023/cont_boiler_unit1.csv')

In [25]:
for group in eps_client.get_groups():
    eps_client.delete_group(group_name = group.name)

INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_nphr_03272023_rev_7_bucket_LB2_6.25_7.5 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_nphr_03272023_rev_7_bucket_LB3_7.5_8.75 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_nphr_03272023_rev_7_bucket_LB4_8.75_10 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_nphr_03272023_rev_7_bucket_LB5_10_12.5 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_nphr_03272023_rev_7_bucket_LB1_5_6.25 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_03272023_rev_1_bucket_LB1_5_6.25 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_03272023_rev_2_bucket_LB2_6.25_7.5 deleted successfully
INFO:sparkbeyond.predictionserver.api.client:Group NPHR_Unit1_03272023_rev_3_bucket_LB3_7.5_8.75 deleted successfully
INFO:sparkbeyond.predictionserver.api

In [16]:
group_name = 'NPHR_Unit1_03272023_rev_1_bucket_LB1_5_6.25'
predict_df_subset = predict_df[(predict_df['load']>=5) & (predict_df['load']<6.25)]

In [17]:
predict_df_subset = predict_df_subset[['datetime', 'load']]

In [18]:
group_placeholder = eps_client.get_group(group_name)

In [19]:
group_placeholder.context_inputs[0].columns

Index(['datetime (yy-MM-dd HH:mm:ss)',
       'PCS05_APP1.C_TOTALAIRFLOW.OUT_TOTAL AIR FLOW CURRENT_ton_per_hr',
       'PCS03_APP1.GAI10HBK01CP001_INLET BACKPASS PRESSURE_mbar',
       'PCS01_APP1.GAI10HBK01CT031_BACKPASS TEMP AFTER LT SH_C',
       'PCS01_APP1.GAI10HBK01CT021_BACKPASS TEMP AFTER INT SH_C',
       'PCS03_APP1.GAI10LBA10CP002_MAIN STEAM PRESSURE_barg',
       'PCS03_APP1.GAI10HBK01CP002_INLET BACKPASS PRESSURE_mbar',
       'PCS01_APP1.GAI10HNA41CT001_ESP INLET TEMP_C',
       'PCS03_APP1.GAI10LBA10CT001_TEMPRATURE AFTER FINAL SH_C',
       'PCS01_APP1.GAI10LAE20CF001_DESH-1 SPRAY WATER FLOW_mmH2O',
       'PCS01_APP1.GAI10HBK01CT042_BACKPASS TEMP AFTER ECO_C',
       'PCS03_APP1.GAI10LBA10CT003_TEMPRATURE AFTER FINAL SH_C',
       'PCS05_APP1.C_HLA25CF901.OUT_SECONDARY AIR FLOW CURRENT_ton_per_hr',
       'PCS03_APP1.GAI10LBA10CT002_TEMPRATURE AFTER FINAL SH_C',
       'PCS01_APP1.GAI10HAC90CT001_INLET FEED WATER TEMP_C',
       'PCS03_APP1.GAI10LBA10CP003_MAIN STEAM 

In [21]:
predict_results = eps_client.predict_with_quality_report(input_df=predict_df,
                                    context_input_df=context_df,
                                    model=group_name,
                                    include_enriched=False,
                                    include_originals=False,
                                    strict_input_validation=False,
                                    context_matching_report=True)


INFO:sparkbeyond.predictionserver.api.client:Sending Predict to https://192.168.106.199/
INFO:sparkbeyond.predictionserver.api.client:https://192.168.106.199/predbox/groups/predict?strictInputValidation=false&qualityReportEnabled=true&contextMatchingReport=true
INFO:sparkbeyond.predictionserver.api.client:Request completed successfully


In [ ]:
# EMA
rolling_mean = load.set_index('datetime')['load'].ewm(span=rolling_window).mean().shift(12).rename(f'ma_t={rolling_window}')